In [1]:
#Import the required packages

import pandas as pd
import os
import nltk, re, pprint
import gensim
import numpy
import csv
import fuzzywuzzy
import time
from datetime import timedelta
from nltk import word_tokenize
from nltk.corpus import stopwords
set(stopwords.words('english'))
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer



In [2]:
#Setting up the working directory


#Print current working directory
print(os.getcwd())

#Change working directory to desired location
os.chdir("/Users/preshantjoseph/Documents/University/Data Science Master Class 1/Assignment 4")

#Confirm working directory has been changed to desired locatio
print(os.getcwd())

/Users/preshantjoseph/Documents/University/Data Science Master Class 1/Assignment 4
/Users/preshantjoseph/Documents/University/Data Science Master Class 1/Assignment 4


In [3]:
#Import field of occupation scrape for inputting to model

data = pd.read_csv("occupations.csv", encoding = 'latin1', header = 0)

occ_data = data

data.head(10)

,page,occ_title,occ_text
0,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,MAJOR GROUP 1 MANAGERS\n,"\nMANAGERS plan, organise, direct, control, co..."
1,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,"SUB-MAJOR GROUP 11 CHIEF EXECUTIVES, GENERAL M...","\nCHIEF EXECUTIVES, GENERAL MANAGERS AND LEGIS..."
2,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,"MINOR GROUP 111 CHIEF EXECUTIVES, GENERAL MANA...","\nCHIEF EXECUTIVES, GENERAL MANAGERS AND LEGIS..."
3,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,UNIT GROUP 1111 CHIEF EXECUTIVES AND MANAGING ...,\nCHIEF EXECUTIVES AND MANAGING DIRECTORS dete...
4,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,UNIT GROUP 1112 GENERAL MANAGERS\n,"\nGENERAL MANAGERS plan, organise, direct, con..."
5,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,UNIT GROUP 1113 LEGISLATORS\n,\nLEGISLATORS represent the interests of peopl...
6,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,SUB-MAJOR GROUP 12 FARMERS AND FARM MANAGERS\n,"\nFARMERS AND FARM MANAGERS plan, organise, co..."
7,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,MINOR GROUP 121 FARMERS AND FARM MANAGERS\n,"\nFARMERS AND FARM MANAGERS plan, organise, co..."
8,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,UNIT GROUP 1211 AQUACULTURE FARMERS\n,"\nAQUACULTURE FARMERS plan, organise, control,..."
9,https://www.abs.gov.au/ausstats/abs@.nsf/Lates...,UNIT GROUP 1212 CROP FARMERS\n,"\nCROP FARMERS plan, organise, control, coordi..."


In [4]:
#Checking for duplicate rows using the page urls 

#1: print all rows where the ID is one of the IDs in duplicated:

URL = data["page"]
Duplicates = data[URL.isin(URL[URL.duplicated()])]

pd.set_option('display.max_colwidth', -1)

Duplicates

,page,occ_title,occ_text
112,https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/B2B7F0A9B46005E3CA2584A8000E7A83?opendocument,UNIT GROUP 2341 AGRICULTURAL AND FORESTRY SCIENTISTS\n,"\nAGRICULTURAL AND FORESTRY SCIENTISTS advise farmers, rural industries and government on aspects of farming, develop techniques for increasing productivity, and study and develop plans and policies for the management of forest areas.\nIndicative Skill Level:\nIn Australia and New Zealand:\nMost occupations in this unit group have a level of skill commensurate with a bachelor degree or higher qualification. In some instances relevant experience and/or on-the-job training may be required in addition to the formal qualification (ANZSCO Skill Level 1).\n\nTasks Include:\n\ncollecting and analysing data and samples of produce, feed, soil and other factors affecting production \nadvising Farmers and Farm Managers on techniques for improving the production of crops and livestock, and alternative agricultural options \nadvising farmers on issues such as livestock and crop disease, control of pests and weeds, soil improvement, animal husbandry and feeding programs \nstudying the environmental factors affecting commercial crop production, pasture growth, animal breeding, and the growth and health of forest trees \nstudying the effects of cultivation techniques, soils, insects and plant diseases on animal, crop and forest production \ndeveloping procedures and techniques for solving agricultural problems and improving the efficiency of production \nmanaging forest resources to maximise their long-term commercial, recreational and environmental benefits for the community \nstudying the propagation and culture of forest trees, methods for improving the growth of stock, and the effects of thinning on forest yields \npreparing plans for reafforestation and devising efficient harvesting systems \ninvestigating, planning and implementing management procedures to cope with the effects of fires, floods, droughts, soil erosion, insect pests and diseases\n\nOccupations:\n234111 Agricultural Consultant \n234112 Agricultural Scientist \n234113 Forester / Forest Scientist\n"
113,https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/B2B7F0A9B46005E3CA2584A8000E7A83?opendocument,Alternative Title:\nAgricultural Adviser\n,"\nStudies commercial plants, animals and cultivation techniques to enhance the productivity of farms and agricultural industries.\nSkill Level: 1\nSpecialisation:\nAgronomist\n"
114,https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/B2B7F0A9B46005E3CA2584A8000E7A83?opendocument,"Advises farmers, agricultural businesses, rural industries and government on the production, processing and distribution of farm products.\n","\nStudies, develops and manages forest areas to maintain commercial and recreational uses, conserve flora and fauna, and protect against fire, pests and diseases.\nSkill Level: 1\nSpecialisations:\nForestry Adviser \nForestry Consultant \n"
331,https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/25A2FCFBA49A8022CA2584A8000E7A1A?opendocument,"UNIT GROUP 4523 SPORTS COACHES, INSTRUCTORS AND OFFICIALS \n\nSPORTS COACHES, INSTRUCTORS AND OFFICIALS coach, train and instruct participants in sports, and officiate at sporting events. \n",\nIndicative Skill Level:\nMost occupations in this unit group have a level of skill commensurate with the qualifications and experience outlined below.\n
332,https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/25A2FCFBA49A8022CA2584A8000E7A1A?opendocument,Tasks Include: \n,"AQF Certificate III including at least two years of on-the-job training, or AQF Certificate IV or at least three years of relevant experience (ANZSCO Skill Level 3)"


In [5]:
#Fix UNIT GROUP 2341 AGRICULTURAL AND FORESTRY SCIENTISTS record by concatenating occ text

#select duplicate records related to unit group 2341
occ_text_112 = Duplicates[Duplicates['page'] =="https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/B2B7F0A9B46005E3CA2584A8000E7A83?opendocument"]

#Select first record to be base
occ_text_112_fix = occ_text_112.iloc[0,:]

#Combine the occupation text of all three records
occ_text_112_fix['occ_text'] = occ_text_112.iloc[0,2] + occ_text_112.iloc[1,2] + occ_text_112.iloc[2,2]

In [6]:
#Fix UNIT GROUP 4523 SPORTS COACHES, INSTRUCTORS AND OFFICIALS record by concatenating occ text

#select duplicate records related to unit group 4523
occ_text_331 = Duplicates[Duplicates['page'] =="https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/25A2FCFBA49A8022CA2584A8000E7A1A?opendocument"]

#Select first record to be base
occ_text_331_fix = occ_text_331.iloc[0,:]

#Combine the occupation text of both records
occ_text_331_fix['occ_text'] = occ_text_331.iloc[0,2] + occ_text_331.iloc[1,2]

In [7]:
#Remove duplicate data and reinstert fixed data

#keep the 2341 combined data
keepdata = pd.DataFrame(occ_text_112_fix).T

#Append with combined 4523 data
keepdata = keepdata.append(occ_text_331_fix)

#drop duplicate rows from data set

#drop rows which have 2341 data 
occ_data = occ_data[occ_data.page != 'https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/B2B7F0A9B46005E3CA2584A8000E7A83?opendocument']

#drop rows which have 4523 data 
occ_data = occ_data[occ_data.page != 'https://www.abs.gov.au/ausstats/abs@.nsf/Latestproducts/25A2FCFBA49A8022CA2584A8000E7A1A?opendocument']

#append occ data with fixed rows for occupations 2341 and 4523
occ_data = occ_data.append(keepdata)